In [1]:
import sqlite3
import pandas as pd

In [2]:
#Dosyaları Dataframe formatına çevirelim
pit_stops = pd.read_csv("pit_stops.csv")
drivers = pd.read_csv("drivers.csv")
constructors = pd.read_csv("constructors.csv")
races = pd.read_csv("races.csv")
results = pd.read_csv("results.csv")

In [3]:
#SQLite bağlantısını yapalım
conn = sqlite3.connect("f1_data.db")

In [4]:
pit_stops.to_sql("pit_stops", conn, if_exists="replace", index=False)
results.to_sql("results", conn, if_exists="replace", index=False)
races.to_sql("races", conn, if_exists="replace",index=False )
constructors.to_sql("constructors", conn, if_exists="replace", index=False)
drivers.to_sql("drivers", conn, if_exists="replace", index=False)


859

In [5]:
query = """
SELECT r.raceId, r.year, r.round, r.circuitId, cir.name AS circuit_name, 
       d.driverId, c.constructorId, ps.milliseconds AS pit_stop_time, 
       res.positionOrder, res.grid, res.points, res.laps, res.statusId
FROM results res
JOIN races r ON res.raceId = r.raceId
LEFT JOIN pit_stops ps ON res.raceId = ps.raceId AND res.driverId = ps.driverId
JOIN drivers d ON res.driverId = d.driverId
JOIN constructors c ON res.constructorId = c.constructorId
JOIN circuits cir ON r.circuitId = cir.circuitId

WHERE r.year >= strftime('%Y', 'now') - 10

"""


In [6]:
#SQL den veriyi okuyalım
data = pd.read_sql_query(query,conn)

#EDA yaparken eksik değer olduğunu fark etmiştik son 10 yılda bu değerler var mı kontrol edip temizleyelim
data = data.dropna(subset=["pit_stop_time"])

#TAHMİN İÇİN HEDEF DEĞİŞKEN GEREKLİ (KAZANDI = 1, KAYBETTİ = 0)
#data['winner'] = (data['positionOrder'] == 1).astype(int)

# Gereksiz sütunları at
#data = data.drop(columns=["positionOrder"])
data=data.drop(columns=["circuit_name"])

In [7]:
#datamızın ilk 5 sütununu kontrol ettik. yıla göre filtreleme yaptığımızı doğruladık. şimdi bu data için EDA.ipynb DOSYASINA TEKRAR GEÇELİM

data.head()

,raceId,year,round,circuitId,driverId,constructorId,pit_stop_time,positionOrder,grid,points,laps,statusId
1,900,2014,1,1,3,131,23673.0,1,3,25.0,57,1
2,900,2014,1,1,3,131,24331.0,1,3,25.0,57,1
3,900,2014,1,1,4,6,21978.0,4,5,12.0,57,1
4,900,2014,1,1,4,6,22887.0,4,5,12.0,57,1
5,900,2014,1,1,8,6,21825.0,7,11,6.0,57,1


In [8]:
#oluşturduğumuz "data" dataframini kaydedelim
data.to_csv('data.csv', index=False)